<a href="https://colab.research.google.com/github/Benchamin242/Scraping/blob/main/Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [4]:
from google.colab import files
uploaded = files.upload()


Saving matches.csv to matches.csv


In [6]:
matches = pd.read_csv("matches.csv", index_col=0) #reading in files, specifying 1st column is index

In [7]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,15/08/2021,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,Match Report,NaN,18,4,16.9,1,0,0,2022,Manchester City
2,21/08/2021,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,Match Report,NaN,16,4,17.3,1,0,0,2022,Manchester City
3,28/08/2021,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,Match Report,NaN,25,10,14.3,0,0,0,2022,Manchester City
4,11/09/2021,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,Match Report,NaN,25,8,14.0,0,0,0,2022,Manchester City
6,18/09/2021,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,...,Match Report,NaN,16,1,15.7,1,0,0,2022,Manchester City


In [8]:
matches.shape

(1389, 27)

In [12]:
matches["team"].value_counts()

,count
team,
Southampton,72
Brighton and Hove Albion,72
Manchester United,72
West Ham United,72
Newcastle United,72
Burnley,71
Leeds United,71
Crystal Palace,71
Manchester City,71


In [13]:
matches["round"].value_counts()

,count
round,
Matchweek 1,39
Matchweek 16,39
Matchweek 34,39
Matchweek 32,39
Matchweek 31,39
Matchweek 29,39
Matchweek 28,39
Matchweek 26,39
Matchweek 25,39


In [14]:
#fewer matches due to scraping mid season

In [15]:
matches.dtypes

,0
date,object
time,object
comp,object
round,object
day,object
venue,object
result,object
gf,int64
ga,int64
opponent,object


In [18]:
matches["date"] = pd.to_datetime(matches["date"])

In [20]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes #away games is 0, home games is 1

In [21]:
matches["opp_code"] = matches['opponent'].astype("category").cat.codes

In [22]:
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int") #replaces minutes in the time and only shows hour as int

In [24]:
matches["day_code"] = matches["date"].dt.dayofweek #converts days of week to int

In [25]:
matches["target"] = (matches["result"] == "W" ).astype("int") #returns boolean, true if win, false if wrong

In [26]:
from sklearn.ensemble import RandomForestClassifier #type of ml model that picks up non liearities in data

In [27]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [28]:
train = matches[matches["date"] < '2022-01-01'] #splitting data from training set to test set

In [29]:
test = matches[matches["date"]> '2022-01-01'] #data after this date used for test set

In [30]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [31]:
rf.fit(train[predictors], train["target"]) #train rf model with predictors to try and train

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [32]:
preds = rf.predict(test[predictors])

In [33]:
from sklearn.metrics import accuracy_score #finding out how often predictions were correct

In [34]:
acc = accuracy_score(test["target"], preds ) #comparing target vs predictions

In [35]:
acc

0.6123188405797102

In [39]:
combined = pd.DataFrame(dict(actual=test["target"], prediction = preds)) #combining our prediction and reality

In [40]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,141,31
1,76,28


In [42]:
from sklearn.metrics import precision_score

In [43]:
precision_score(test["target"],preds) #will show us precision, when predicting win, how often its correct

0.4745762711864407

In [54]:
grouped_matches = matches.groupby("team")

In [56]:
group = grouped_matches.get_group("Manchester City").sort_values("date")


In [57]:
def rolling_averages(group, cols, new_cols):
  group = group.sort_values("date")
  rolling_stats = group[cols].rolling(3, closed='left').mean()
  group[new_cols] = rolling_stats
  group = group.dropna(subset=new_cols)
  return group
  #defines rolling averages of previous 3 games

In [58]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

In [59]:
new_cols #the new rows we created for rolling averages

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [68]:
rolling_averages(group, cols, new_cols)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
5,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Home,W,1,0,Arsenal,...,5,1,2.000000,2.333333,17.333333,4.666667,18.900000,1.333333,0.333333,0.333333
7,2020-10-24,12:30,Premier League,Matchweek 6,Sat,Away,D,1,1,West Ham,...,5,0,1.333333,2.000000,17.333333,3.666667,17.733333,0.666667,0.000000,0.000000
9,2020-10-31,12:30,Premier League,Matchweek 7,Sat,Away,W,1,0,Sheffield Utd,...,5,1,1.000000,0.666667,16.666667,4.333333,18.233333,0.666667,0.000000,0.000000
11,2020-11-08,16:30,Premier League,Matchweek 8,Sun,Home,D,1,1,Liverpool,...,6,0,1.000000,0.333333,14.333333,6.666667,18.466667,1.000000,0.000000,0.000000
12,2020-11-21,17:30,Premier League,Matchweek 9,Sat,Away,L,0,2,Tottenham,...,5,0,1.000000,0.666667,12.000000,5.666667,19.366667,1.000000,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2022-03-14,20:00,Premier League,Matchweek 29,Mon,Away,D,0,0,Crystal Palace,...,0,0,2.333333,1.333333,19.000000,7.000000,15.366667,0.333333,0.333333,0.333333
44,2022-04-02,15:00,Premier League,Matchweek 31,Sat,Away,W,2,0,Burnley,...,5,1,1.666667,0.333333,18.333333,7.333333,16.000000,0.333333,0.000000,0.000000
46,2022-04-10,16:30,Premier League,Matchweek 32,Sun,Home,D,2,2,Liverpool,...,6,0,2.000000,0.333333,20.000000,6.666667,16.133333,0.333333,0.000000,0.000000
49,2022-04-20,20:00,Premier League,Matchweek 30,Wed,Home,W,3,0,Brighton,...,2,1,1.333333,0.666667,15.666667,4.666667,16.700000,0.333333,0.000000,0.000000


In [69]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
#applied function to get rolling averages for every match

In [70]:
matches_rolling

date   time            comp         round  \
team                                                                         
Arsenal                 6  2020-10-04  14:00  Premier League   Matchweek 4   
                        7  2020-10-17  17:30  Premier League   Matchweek 5   
                        9  2020-10-25  19:15  Premier League   Matchweek 6   
                        11 2020-11-01  16:30  Premier League   Matchweek 7   
                        13 2020-11-08  19:15  Premier League   Matchweek 8   
...                               ...    ...             ...           ...   
Wolverhampton Wanderers 32 2022-03-13  14:00  Premier League  Matchweek 29   
                        33 2022-03-18  20:00  Premier League  Matchweek 30   
                        34 2022-04-02  15:00  Premier League  Matchweek 31   
                        35 2022-04-08  20:00  Premier League  Matchweek 32   
                        36 2022-04-24  14:00  Premier League  Matchweek 34   

                            day venue result  gf  ga         opponent  ...  \
team                                                                   ...   
Arsenal                 6   Sun  Home      W   2   1    Sheffield Utd  ...   
                        7   Sat  Away      L   0   1  Manchester City  ...   
                        9   Sun  Home      L   0   1   Leicester City  ...   
                        11  Sun  Away      W   1   0   Manchester Utd  ...   
                        13  Sun  Home      L   0   3      Aston Villa  ...   
...                         ...   ...    ...  ..  ..              ...  ...   
Wolverhampton Wanderers 32  Sun  Away      W   1   0          Everton  ...   
                        33  Fri  Home      L   2   3     Leeds United  ...   
                        34  Sat  Home      W   2   1      Aston Villa  ...   
                        35  Fri  Away      L   0   1    Newcastle Utd  ...   
                        36  Sun  Away      L   0   1          Burnley  ...   

                            day_code  target  gf_rolling  ga_rolling  \
team                                                                   
Arsenal                 6          6       1    2.000000    1.333333   
                        7          5       0    1.666667    1.666667   
                        9          6       0    1.000000    1.666667   
                        11         6       1    0.666667    1.000000   
                        13         6       0    0.333333    0.666667   
...                              ...     ...         ...         ...   
Wolverhampton Wanderers 32         6       1    1.333333    1.000000   
                        33         4       0    1.666667    0.666667   
                        34         5       1    2.333333    1.000000   
                        35         4       0    1.666667    1.333333   
                        36         6       0    1.333333    1.666667   

                           sh_rolling sot_rolling dist_rolling fk_rolling  \
team                                                                        
Arsenal                 6    7.666667    3.666667    14.733333   0.666667   
                        7    5.333333    3.666667    15.766667   0.000000   
                        9    7.000000    3.666667    16.733333   0.666667   
                        11   9.666667    4.000000    16.033333   1.000000   
                        13   9.666667    2.666667    18.033333   1.000000   
...                               ...         ...          ...        ...   
Wolverhampton Wanderers 32  12.333333    3.666667    19.300000   0.000000   
                        33  12.333333    4.333333    19.600000   0.000000   
                        34  13.000000    5.333333    19.833333   0.000000   
                        35  13.000000    5.000000    18.533333   0.000000   
                        36  10.000000    4.666667    17.633333   0.000000   

                            pk_rolling  pkatt_rolling  
team    

In [71]:
matches_rolling = matches_rolling.droplevel('team')


In [72]:
matches_rolling.index = range(matches_rolling.shape[0]) #give unqiue indice for each line

In [73]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2,1,Sheffield Utd,...,6,1,2.000000,1.333333,7.666667,3.666667,14.733333,0.666667,0.000000,0.000000
1,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0,1,Manchester City,...,5,0,1.666667,1.666667,5.333333,3.666667,15.766667,0.000000,0.000000,0.000000
2,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0,1,Leicester City,...,6,0,1.000000,1.666667,7.000000,3.666667,16.733333,0.666667,0.000000,0.000000
3,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1,0,Manchester Utd,...,6,1,0.666667,1.000000,9.666667,4.000000,16.033333,1.000000,0.000000,0.000000
4,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0,3,Aston Villa,...,6,0,0.333333,0.666667,9.666667,2.666667,18.033333,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,2022-03-13,14:00,Premier League,Matchweek 29,Sun,Away,W,1,0,Everton,...,6,1,1.333333,1.000000,12.333333,3.666667,19.300000,0.000000,0.000000,0.000000
1313,2022-03-18,20:00,Premier League,Matchweek 30,Fri,Home,L,2,3,Leeds United,...,4,0,1.666667,0.666667,12.333333,4.333333,19.600000,0.000000,0.000000,0.000000
1314,2022-04-02,15:00,Premier League,Matchweek 31,Sat,Home,W,2,1,Aston Villa,...,5,1,2.333333,1.000000,13.000000,5.333333,19.833333,0.000000,0.000000,0.000000
1315,2022-04-08,20:00,Premier League,Matchweek 32,Fri,Away,L,0,1,Newcastle Utd,...,4,0,1.666667,1.333333,13.000000,5.000000,18.533333,0.000000,0.000000,0.000000


In [86]:
def make_predictions(data, predictors):
  train = data[data["date"] < '2022-01-01']
  test = data[data["date"]> '2022-01-01']
  rf.fit(train[predictors], train["target"])
  preds = rf.predict(test[predictors])
  combined = pd.DataFrame(dict(actual=test["target"], prediction = preds), index = test.index)
  precision = precision_score(test["target"], preds)
  return combined, precision

In [87]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [88]:
precision

0.625

In [89]:
combined

,actual,prediction
55,0,0
56,1,0
57,1,0
58,1,1
59,1,1
...,...,...
1312,1,0
1313,0,0
1314,1,0
1315,0,0


In [98]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)
#merging based on index


In [99]:
combined

,actual,prediction,date_x,team_x,opponent_x,result_x,date_y,team_y,opponent_y,result_y,date,team,opponent,result
55,0,0,2022-01-23,Arsenal,Burnley,D,2022-01-23,Arsenal,Burnley,D,2022-01-23,Arsenal,Burnley,D
56,1,0,2022-02-10,Arsenal,Wolves,W,2022-02-10,Arsenal,Wolves,W,2022-02-10,Arsenal,Wolves,W
57,1,0,2022-02-19,Arsenal,Brentford,W,2022-02-19,Arsenal,Brentford,W,2022-02-19,Arsenal,Brentford,W
58,1,1,2022-02-24,Arsenal,Wolves,W,2022-02-24,Arsenal,Wolves,W,2022-02-24,Arsenal,Wolves,W
59,1,1,2022-03-06,Arsenal,Watford,W,2022-03-06,Arsenal,Watford,W,2022-03-06,Arsenal,Watford,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W,2022-03-13,Wolverhampton Wanderers,Everton,W,2022-03-13,Wolverhampton Wanderers,Everton,W
1313,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L,2022-03-18,Wolverhampton Wanderers,Leeds United,L,2022-03-18,Wolverhampton Wanderers,Leeds United,L
1314,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,2022-04-02,Wolverhampton Wanderers,Aston Villa,W
1315,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L


In [100]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton",
              "Manchester United": "Manchester Utd",
              "Newcastle United": "Newcastle Utd",
              "Tottenham Hotspur": "Tottenham",
              "West Ham United": "West Ham",
              "Wolverhampton Wanderers": "Wolves"}

mapping = MissingDict(**map_values)

In [101]:
mapping["West Ham United"]

'West Ham'

In [102]:
combined["new_team"] = combined["team"].map(mapping)

In [103]:
combined


,actual,prediction,date_x,team_x,opponent_x,result_x,date_y,team_y,opponent_y,result_y,date,team,opponent,result,new_team
55,0,0,2022-01-23,Arsenal,Burnley,D,2022-01-23,Arsenal,Burnley,D,2022-01-23,Arsenal,Burnley,D,Arsenal
56,1,0,2022-02-10,Arsenal,Wolves,W,2022-02-10,Arsenal,Wolves,W,2022-02-10,Arsenal,Wolves,W,Arsenal
57,1,0,2022-02-19,Arsenal,Brentford,W,2022-02-19,Arsenal,Brentford,W,2022-02-19,Arsenal,Brentford,W,Arsenal
58,1,1,2022-02-24,Arsenal,Wolves,W,2022-02-24,Arsenal,Wolves,W,2022-02-24,Arsenal,Wolves,W,Arsenal
59,1,1,2022-03-06,Arsenal,Watford,W,2022-03-06,Arsenal,Watford,W,2022-03-06,Arsenal,Watford,W,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W,2022-03-13,Wolverhampton Wanderers,Everton,W,2022-03-13,Wolverhampton Wanderers,Everton,W,Wolves
1313,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L,2022-03-18,Wolverhampton Wanderers,Leeds United,L,2022-03-18,Wolverhampton Wanderers,Leeds United,L,Wolves
1314,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,Wolves
1315,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,Wolves


In [107]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])
#look for new team and merge with opponent on other side


In [108]:
merged

,actual_x,prediction_x,date_x_x,team_x_x,opponent_x_x,result_x_x,date_y_x,team_y_x,opponent_y_x,result_y_x,...,opponent_x_y,result_x_y,date_y_y,team_y_y,opponent_y_y,result_y_y,team_y,opponent_y,result_y,new_team_y
0,0,0,2022-01-23,Arsenal,Burnley,D,2022-01-23,Arsenal,Burnley,D,...,Arsenal,D,2022-01-23,Burnley,Arsenal,D,Burnley,Arsenal,D,Burnley
1,1,0,2022-02-10,Arsenal,Wolves,W,2022-02-10,Arsenal,Wolves,W,...,Arsenal,L,2022-02-10,Wolverhampton Wanderers,Arsenal,L,Wolverhampton Wanderers,Arsenal,L,Wolves
2,1,0,2022-02-19,Arsenal,Brentford,W,2022-02-19,Arsenal,Brentford,W,...,Arsenal,L,2022-02-19,Brentford,Arsenal,L,Brentford,Arsenal,L,Brentford
3,1,1,2022-02-24,Arsenal,Wolves,W,2022-02-24,Arsenal,Wolves,W,...,Arsenal,L,2022-02-24,Wolverhampton Wanderers,Arsenal,L,Wolverhampton Wanderers,Arsenal,L,Wolves
4,1,1,2022-03-06,Arsenal,Watford,W,2022-03-06,Arsenal,Watford,W,...,Arsenal,L,2022-03-06,Watford,Arsenal,L,Watford,Arsenal,L,Watford
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W,2022-03-13,Wolverhampton Wanderers,Everton,W,...,Wolves,L,2022-03-13,Everton,Wolves,L,Everton,Wolves,L,Everton
258,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L,2022-03-18,Wolverhampton Wanderers,Leeds United,L,...,Wolves,W,2022-03-18,Leeds United,Wolves,W,Leeds United,Wolves,W,Leeds United
259,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,...,Wolves,L,2022-04-02,Aston Villa,Wolves,L,Aston Villa,Wolves,L,Aston Villa
260,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,...,Wolves,W,2022-04-08,Newcastle United,Wolves,W,Newcastle United,Wolves,W,Newcastle Utd


In [111]:
merged[(merged["prediction_x"] == 1) & (merged["prediction_y"] ==0)]["actual_x"].value_counts()


,count
actual_x,
1,27
0,13


In [112]:
27/40

0.675

In [114]:
#improved accuracy to 67%